<a href="https://colab.research.google.com/github/yugonojima/NLP100Exercise-Chapter10/blob/main/Chapter10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow-addons==0.11.2
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 32.6 MB/s 


In [ ]:
%cd /content/drive/MyDrive/NLP100本ノック/Chapter10/

/content/drive/MyDrive/NLP100本ノック/Chapter10


##チャレンジコース第１０章


使用したモデル：seq2seq(encoder:LSTM, decoder:LSTM with attention_mechanism)

工夫した前処理

・形態素解析させた日本語テキストの逆順

・英語テキストの小文字化

・(出現回数が一定以下の形態素をマスク)

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import unicodedata
import re
import numpy as np
import os
import io
import time

In [ ]:
train_en_path = './tok/kyoto-train.en'
train_ja_path = './tok/kyoto-train.ja'
train_cln_en_path = './tok/kyoto-train.cln.en'
train_cln_ja_path = './tok/kyoto-train.cln.ja'
tune_en_path = './tok/kyoto-tune.en'
tune_ja_path = './tok/kyoto-tune.ja'
dev_en_path = './tok/kyoto-dev.en'
dev_ja_path = './tok/kyoto-dev.ja'
test_en_path = './tok/kyoto-test.en'
test_ja_path = './tok/kyoto-test.ja'

seed = 0

In [ ]:
class NMTDataset:
    def __init__(self, problem_type='ja-en'):
        self.problem_type = 'ja_en'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None

    def inp_Preprocess(self, text):
        text = text.replace('\n', '')
        # text = list(reversed(text.split()))
        # text = ' '.join(text)
        text = "<start> "+text+" <end>"
        return text
    
    def targ_Preprocess(self, text):
        text = text.replace('\n', '')
        # text = text.lower()
        text = '<start> '+text+' <end>'
        return text

    def remove_word_listup(self, lst, value):#value = 出現頻度の閾値
            return [i for i, x in enumerate(lst) if x <= value]

    def remove_Fewer_appearances(self, texts, tok):
        appearances_list = [0]*(len(tok.word_index))
        for text in texts:
            text = text.lower()
            text = text.split()
            for word in text:
                appearances_list[tok.word_index[word]-2] += 1

        remove_list = self.remove_word_listup(appearances_list, 5)
        remove_list = [k for k, v in tok.word_index.items() if v in remove_list]
        removed_lines =[]
        for text in texts:
            text = text.lower()
            split_text = text.split()
            for word in split_text:
                if word in remove_list:
                    text = text.replace(word, '<unk>')
            removed_lines.append(text) 
        return removed_lines  



    def Read(self, path, num_examples=None, inp = False):
        new_lines = []
        with open(path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            if inp:
                new_lines.append(self.inp_Preprocess(line))
            else:
                new_lines.append(self.targ_Preprocess(line))
        if num_examples == None:
            return new_lines
        else:
            return new_lines[:num_examples]

    def tokenize(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='\n', oov_token="<unk>")
        tokenizer.fit_on_texts(texts)

        # texts = self.remove_Fewer_appearances(texts, tokenizer)
        # tokenizer.fit_on_texts(texts)

        tensor = tokenizer.texts_to_sequences(texts)
        tensor = pad_sequences(tensor, padding="post")

        return  tensor, tokenizer

    def load_dataset(self, input_path, tar_path , num_examples=None):
        # creating cleaned input, output pairs
        input_texts = self.Read(input_path, num_examples, inp = True)
        target_texts = self.Read(tar_path, num_examples)

        input_tensor, input_tokenizer = self.tokenize(input_texts)
        target_tensor, target_tokenizer = self.tokenize(target_texts)



        return input_tensor, target_tensor, input_tokenizer, target_tokenizer


    def call(self, ja_path , en_path,  num_examples, BUFFER_SIZE, BATCH_SIZE):
    
        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(ja_path, en_path, num_examples)

        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE, seed=seed).batch(BATCH_SIZE, drop_remainder=True)#30000*0.8/64 = 375data

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [ ]:
# #前処理の挙動の確認のセル
# import unicodedata

# def is_japanese(string):
#     for ch in string:
#         name = unicodedata.name(ch) 
#         if "CJK UNIFIED" in name \
#         or "HIRAGANA" in name \
#         or "KATAKANA" in name:
#             return True
#     return False

# new_lines = []
# reverse_lines = []
# include_ja_text = []

# with open(train_cln_ja_path, 'r') as f:
#     lines = f.readlines()
# for line in lines[:1]:
#     print(line)
#     print(len(line))
#     # reverse_line = list(reversed(line.split()))
#     reverse_line = ' '.join(line)
#     print(reverse_line)
#     print(len(reverse_line))
#     # reverse_line = "start "+reverse_line+" end"
#     # reverse_lines.append(reverse_line)
#     # if is_japanese(line) :
#     #     include_ja_text.append(line)
#     text = "<start> "+line+" <end>"
#     new_lines.append(text)

# print(new_lines)
# # print(reverse_lines[0])



In [ ]:
BUFFER_SIZE = 100000 #データからここで定義した要素分取り出し、その中でシャッフルする
BATCH_SIZE = 64
# Let's limit the #training examples for faster training
num_examples = 100000

dataset_creator = NMTDataset('ja_en')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(train_cln_ja_path, train_cln_en_path, num_examples, BUFFER_SIZE, BATCH_SIZE)

In [ ]:
print(inp_lang.word_index)

{'<unk>': 1, '<start>': 2, '<end>': 3, 'の': 4, '。': 5, '、': 6, 'に': 7, 'る': 8, 'は': 9, 'を': 10, 'と': 11, 'が': 12, 'い': 13, 'て': 14, 'で': 15, 'た': 16, 'し': 17, '（': 18, '）': 19, 'あ': 20, 'れ': 21, 'な': 22, 'っ': 23, 'も': 24, 'さ': 25, '年': 26, '」': 27, '「': 28, 'する': 29, '・': 30, 'う': 31, 'く': 32, 'こと': 33, 'り': 34, 'から': 35, 'など': 36, 'や': 37, 'もの': 38, 'られ': 39, 'その': 40, '日本': 41, '月': 42, 'よ': 43, 'ら': 44, '日': 45, 'わ': 46, '-': 47, '市': 48, '的': 49, 'この': 50, 'つ': 51, 'また': 52, '時代': 53, '寺': 54, ')': 55, '(': 56, 'よう': 57, 'お': 58, 'ため': 59, '京都': 60, 'す': 61, '県': 62, 'ば': 63, '』': 64, '『': 65, 'これ': 66, 'か': 67, '多': 68, '後': 69, '一': 70, '呼': 71, '2': 72, '1': 73, '行': 74, '：': 75, '3': 76, 'き': 77, 'だ': 78, '現在': 79, '用い': 80, '天皇': 81, '場合': 82, 'ん': 83, '者': 84, '人': 85, '江戸': 86, '4': 87, '中': 88, '第': 89, '家': 90, '三': 91, 'せ': 92, '5': 93, 'まで': 94, '使': 95, '作': 96, 'へ': 97, '文化': 98, '代': 99, '目': 100, '国': 101, '二': 102, 'しかし': 103, '流': 104, '院': 105, '明治': 106, '宗': 107

In [ ]:
example_input_batch, example_target_batch = next(iter(train_dataset))
print(example_input_batch.shape, example_target_batch.shape)
print(len(train_dataset))

(64, 42) (64, 42)
1250


In [ ]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dim = 128 #単語の埋め込み次元数
units = 512 #各セルの出力次元数
steps_per_epoch = int(len(train_dataset)) #iteration
val_steps_per_epoch = int(len(val_dataset))

In [ ]:
print(vocab_inp_size)
print(max_length_input)

65794
42


In [ ]:
##### 

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    ##-------- LSTM layer in Encoder ------- ##
    self.LSTM_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'
                                   )



  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.LSTM_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [ ]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden) #outputにはlstmlayer全体の出力が格納される
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
# print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
# print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type

    # Embedding Layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    #Final Dense layer on which softmax will be applied(softmaxはloss算出時にかけられるため、ここでは記述しない)
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Define the fundamental cell for decoder recurrent structure
    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)



    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler() #decoderの各セルの入出力のアルゴリズムを設定するインスタンス。通常は直接使用する必要はないが、BasicDecoderオブジェクトを使用する際は設定しておかなければならない

    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_input], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
    self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell (使用するセルとsampler, output_layerをコンストラクタとして設定するだけで、簡単にseq2seqのモデルをインスタンス化して作成することができる)
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)


  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)#decoder_rnn_cell.get_initial_state→リスト、rnn_cell.get_initial_state→attentionwrapperのためのタプル
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)#attentionを使用する際は必要
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
    return outputs

In [ ]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)
print(sample_decoder_outputs.sample_id)
print(sample_decoder_outputs.rnn_output[0][0])
print(np.array(tf.reduce_sum(sample_decoder_outputs.rnn_output[0][0])))
# print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)#.rnn_output = decoderで出力した値のrnn_output部分を抜き出す

tf.Tensor(
[[65373 20861 20861 ... 52229 52229 52229]
 [50625 25671 25671 ... 52229 52229 52229]
 [66061 25671 25671 ... 41199 41199 41199]
 ...
 [24255 24255 24255 ... 21886 21886 21886]
 [37922 24255 24255 ... 66210 66210 66210]
 [50625 20861 20861 ... 41199 41199 41199]], shape=(64, 41), dtype=int32)
tf.Tensor(
[ 0.00053666  0.00040271 -0.00010561 ...  0.00045177 -0.0001539
 -0.00142555], shape=(71709,), dtype=float32)
0.2673424


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate =0.001)


def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')#ラベルが整数の際はSparseCategoricalCrossentropyを使用する(ラベルがone_hot_vectorの際はcategorical_crossentropyを使用する)
#   loss = tf.nn.sampled_softmax_loss(weights,biases,labels,inputs,num_sampled,num_classes,num_true=1,sampled_values=None,remove_accidental_hits=True,seed=None,name='sampled_softmax_loss')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))   #tf.math.equal(real,0)→realの0の部分をTrueとしたtensorを返す、tf.logical_not→bool値を逆転させる
  mask = tf.cast(mask, dtype=loss.dtype)  #true→1, false→0にしたtensorを作成
  loss = mask* loss #target配列のpadding処理で0に変換した部分のlossを無視させる
  loss = tf.reduce_mean(loss)#バッチ間での平均をとる
  return loss

In [ ]:
checkpoint_dir = './training_checkpoints2'
checkpoint_prefix = os.path.join(checkpoint_dir, "epoch-10")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function #@tf.functionを付けると、その関数はグラフにコンパイルされ速く動作する
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs (初期値のmemoryはNoneに設定してあるが、attention_mechanismのsetup_memoryメソッドを使用することで、encoderの出力をattentionに使用させている)
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output #logits：softmaxをかける前の出力の値
    loss = loss_function(real, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  gradients = [tf.clip_by_norm(g, 1.0) for g in gradients] #clipping(勾配爆発を防ぐ)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [ ]:
@tf.function #@tf.functionを付けると、その関数はグラフにコンパイルされ速く動作する
def test_step(inp, targ, enc_hidden):
    loss = 0
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs (初期値のmemoryはNoneに設定してあるが、attention_mechanismのsetup_memoryメソッドを使用することで、encoderの出力をattentionに使用させている)
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output 
    loss = loss_function(real, logits)

    return loss

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)

  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    train_batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += train_batch_loss
    total_val_loss = 0
    for (val_batch, (val_inp, val_targ)) in enumerate(val_dataset.shuffle(val_steps_per_epoch).take(2)):
        val_batch_loss = test_step(val_inp, val_targ, enc_hidden)
        total_val_loss += val_batch_loss

    if batch % 1000 == 0:
      print('Epoch {} Batch {} Train_Loss {:.4f} Test_Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   train_batch_loss.numpy(),
                                                   total_val_loss / 2))
#   saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 1 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Train_total_Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Train_Loss 5.0704 Test_Loss 4.9753
Epoch 1 Batch 1000 Train_Loss 2.2489 Test_Loss 2.1346
Epoch 1 Train_total_Loss 2.4298
Time taken for 1 epoch 2022.4166808128357 sec



In [ ]:
# for w in encoder.weights:
#     print('{:<25}{}'.format(w.name, w.shape))
print(encoder.weights[0].numpy())
print('decoder')
print(decoder.weights[0].numpy())

[[ 0.03613368  0.04244112 -0.0478912  ... -0.0129896  -0.02010633
  -0.03035936]
 [ 0.03254377  0.00291363  0.04161019 ...  0.03381354 -0.03892922
  -0.04267139]
 [-0.00393587  0.01972066 -0.04028078 ... -0.00359542 -0.04329139
  -0.01375145]
 ...
 [ 0.01586064  0.04354275  0.04501058 ...  0.03010831 -0.04806193
  -0.0334866 ]
 [-0.04508452 -0.02157786  0.02660282 ... -0.01193342  0.03128023
  -0.00815183]
 [-0.00650947 -0.01557014 -0.00199525 ... -0.02776501 -0.0024853
   0.03360063]]
decoder
[[-0.0449934   0.01358371  0.02462493 ... -0.03899089 -0.04678664
  -0.03980906]
 [ 0.00383254 -0.02813593 -0.02560999 ... -0.00561057 -0.01254381
   0.04980768]
 [-0.0020581  -0.03611472 -0.02504896 ...  0.04919943 -0.04440556
   0.01061653]
 ...
 [-0.04549476  0.02947393 -0.01090997 ... -0.00864886  0.00036826
  -0.01987495]
 [ 0.0431819   0.01536495 -0.01016242 ...  0.04304853 -0.04993037
  -0.04972254]
 [-0.03361358 -0.02426714 -0.03433721 ... -0.037516    0.02775712
   0.02682096]]


In [ ]:
def evaluate_sentence(sentence):
  inputs = inp_lang.texts_to_sequences([sentence])
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                        #   maxlen=max_length_input,
                                                          padding='post')
#   inputs = [inp_lang.word_index[i] for i in sentence.split(' ')] #embedding
#   inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],  #add padding
#                                                         #   maxlen=max_length_input,
#                                                           padding='post')
#   inputs = tf.convert_to_tensor(inputs) #inputs.shape=(1, text_size)
  inference_batch_size = inputs.shape[0]#1
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)#encoderの初期値は学習済みのものを使用しなくてもよいのか？

  dec_h = enc_h
  dec_c = enc_c
#   print(enc_h.shape)
#   print(enc_c.shape)

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
  end_token = targ_lang.word_index['<end>']

  #ここからカスタム

  maximum_iterations = inputs.shape[1] * 2
#   decoder.attention_mechanism.setup_memory(enc_out)
#   decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)
#   decoder_embedding_matrix = decoder.embedding.variables[0]

#   # Helper
#   helper = tfa.seq2seq.GreedyEmbeddingHelper(decoder_embedding_matrix,
#                                                     tf.fill([inference_batch_size], targ_lang.word_index['<start>']), targ_lang.word_index['<end>'])
#   # Decoder   
#   decoder_instance = tfa.seq2seq.BasicDecoder(decoder.rnn_cell, helper, decoder_initial_state, output_layer=decoder.fc)
#   # Dynamic decoding
#   outputs, _, _ = tfa.seq2seq.dynamic_decode(decoder_instance, maximum_iterations=maximum_iterations)

  



  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  # Instantiate BasicDecoder object
  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc, maximum_iterations=maximum_iterations )#samplerをgreedy_samplerにすることで、各セルの入力を一つ前のセルの出力で予測された単語になるように設定している
  # Setup Memory in decoder stack
  decoder.attention_mechanism.setup_memory(enc_out)

  # set decoder_initial_state
  decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  
  return outputs.sample_id.numpy()

def translate(ja_sentence, en_sentence):
  result = evaluate_sentence(ja_sentence)
  print(result)
  result = targ_lang.sequences_to_texts(result)
  ja_sentence = ja_sentence.replace('\n', ' ')
#   ja_sentence = list(reversed(ja_sentence.split()))
#   ja_sentence = ' '.join(ja_sentence)
  print('Input: %s' % (ja_sentence))
  print('Correct Answer: %s' % (en_sentence))
  print('Predicted translation: {}'.format(result))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
checkpoint.restore('/content/drive/MyDrive/NLP100本ノック/Chapter10/training_checkpoints2/epoch-10')

In [ ]:
ja_dev_texts = dataset_creator.Read(train_cln_ja_path, inp=True)
en_dev_texts = dataset_creator.Read(train_cln_en_path)


for ja_sentence, en_sentence in zip(ja_dev_texts[:2], en_dev_texts[:2]):
    translate(ja_sentence, en_sentence)
    # ja_sentence = ja_sentence.replace('\n', '')
    # ja_sentence = list(reversed(ja_sentence.split()))
    # # ja_sentence = ' '.join(ja_sentence)
    # print('Input: %s' % (ja_sentence))


[[   27 36629     4    45   793   492     8    50     6     3]]
Input: <start> 日本 の 水墨 画 を 一変 さ せ た 。 <end>
Correct Answer: <start> He revolutionized the Japanese ink painting . <end>
Predicted translation: ['he revolutionized the japanese ink painting in japan . <end>']
[[  27   16  292    4  681   65   19 5720 5127   19   31   19 3025    6
    19    3]]
Input: <start> 諱 は 「 等楊 （ とうよう ） 」 、 もしくは 「 拙宗 （ せっしゅう ） 」 と 号 し た 。 <end>
Correct Answer: <start> He was given the posthumous name " Toyo " or " Sesshu ( 拙宗 ) . " <end>
Predicted translation: ['he was given the posthumous name " toyo shichijo " or " sesshu . " <end>']


In [ ]:
# ja_dev_texts = dataset_creator.Read(train_cln_ja_path, inp=True)
# en_dev_texts = dataset_creator.Read(train_cln_en_path)
# # ja_dev_texts = dataset_creator.remove_Fewer_appearances(ja_dev_texts[:1000], inp_lang)
# for (batch, (inp, targ)) in enumerate(train_dataset.take(1)):
#     enc_hidden = encoder.initialize_hidden_state()
#     enc_output, enc_h, enc_c = encoder(inp, enc_hidden)

#     dec_input = targ[ : , :-1 ] # Ignore <end> token
#     real = targ[ : , 1: ]         # ignore <start> token

#     # Set the AttentionMechanism object with encoder_outputs (初期値のmemoryはNoneに設定してあるが、attention_mechanismのsetup_memoryメソッドを使用することで、encoderの出力をattentionに使用させている)
#     decoder.attention_mechanism.setup_memory(enc_output)

#     # Create AttentionWrapperState as initial_state for decoder
#     decoder_initial_state = decoder.build_initial_state(1, [enc_h, enc_c], tf.float32)
#     pred = decoder(dec_input, decoder_initial_state)
# ########################################
#     enc_start_state = [tf.zeros((BATCH_SIZE, units)), tf.zeros((BATCH_SIZE,units))]
#     enc_out, enc_h, enc_c = encoder(inp, enc_start_state)#encoderの初期値は学習済みのものを使用しなくてもよいのか？

#     start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
#     end_token = targ_lang.word_index['<end>']

#     maximum_iterations = inputs.shape[1] * 2

#     greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

#     # Instantiate BasicDecoder object
#     decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc, maximum_iterations=maximum_iterations )#samplerをgreedy_samplerにすることで、各セルの入力を一つ前のセルの出力で予測された単語になるように設定している
#     # Setup Memory in decoder stack
#     decoder.attention_mechanism.setup_memory(enc_out)

#     # set decoder_initial_state
#     decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


#     ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
#     ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
#     ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

#     decoder_embedding_matrix = decoder.embedding.variables[0]

#     outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)

#     # translate(ja_sentence, en_sentence)

In [ ]:
def beam_evaluate_sentence(sentence, beam_width=3):

  inputs = inp_lang.texts_to_sequences([sentence])
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                        #   maxlen=max_length_input,
                                                          padding='post')
#   inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])#tf.fill()→第一引数のshapeに、第二引数の値を満たしたtensor型を返す
  end_token = targ_lang.word_index['<end>']
  maximum_iterations = inputs.shape[1] * 2
#   print(f'maximum_iterations : {maximum_iterations}')

  # From official documentation
  # NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:
  # The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
  # The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
  # The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.

  enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
  decoder.attention_mechanism.setup_memory(enc_out)
#   print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  # set decoder_inital_state which is an AttentionWrapperState considering beam_width
  hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
  decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  # Instantiate BeamSearchDecoder
  decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc, maximum_iterations=maximum_iterations)
  decoder_embedding_matrix = decoder.embedding.variables[0]

  # The BeamSearchDecoder object's call() function takes care of everything.
  outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)
  # outputs is tfa.seq2seq.FinalBeamSearchDecoderOutput object. 
  # The final beam predictions are stored in outputs.predicted_id
  # outputs.beam_search_decoder_output is a tfa.seq2seq.BeamSearchDecoderOutput object which keep tracks of beam_scores and parent_ids while performing a beam decoding step
  # final_state = tfa.seq2seq.BeamSearchDecoderState object.
  # Sequence Length = [inference_batch_size, beam_width] details the maximum length of the beams that are generated


  # outputs.predicted_id.shape = (inference_batch_size, time_step_outputs, beam_width)
  # outputs.beam_search_decoder_output.scores.shape = (inference_batch_size, time_step_outputs, beam_width)
  # Convert the shape of outputs and beam_scores to (inference_batch_size, beam_width, time_step_outputs)
  final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

  return final_outputs.numpy(), beam_scores.numpy()

In [ ]:
def beam_translate(inp_sentence, targ_sentence):
  result, beam_scores = beam_evaluate_sentence(inp_sentence)
#   print(result.shape, beam_scores.shape)
  output_list = []
  for beam, score in zip(result, beam_scores):
    # print(beam.shape, score.shape)
    output = targ_lang.sequences_to_texts(beam)
    # print(output)
    for pre_text in output: #予測したテキストに<end>が入っていない場合があるので、それぞれ場合わけした
        pre_splited_text = pre_text.split()
        if '<end>' in pre_splited_text:
            output_list.append(pre_text[:pre_text.index('<end>')])
        else:
            output_list.append(pre_text)
    # output = [a[:a.index('<end>')] for a in output]
    beam_score = [a.sum() for a in score]
    max_score_text = output_list[beam_score.index(max(beam_score))]
    inp_sentence = inp_sentence.replace('\n', ' ')
    # inp_sentence = list(reversed(inp_sentence.split()))
    # inp_sentence = ' '.join(inp_sentence)

    # print('Input: %s' % (inp_sentence))
    # print('Correct Answer: %s' % (targ_sentence))
    # for i , pre_text in enumerate(output_list):
    #   print('{} Predicted translation : {} , beam_socre : {}'.format(i+1, output_list[i], beam_score[i]))
    
    return max_score_text

def Evaluate(ja_texts, en_texts): #BLUE scoreの算出
  score = 0.0
  for ja_text, en_text in zip(ja_texts, en_texts):
    predicted_text = beam_translate(ja_text, en_text)
    score += sentence_bleu([en_text.strip().split()], predicted_text.strip().split(), smoothing_function=SmoothingFunction().method4)#smoothingを入れることによって、precision(i-Gramでの文章の一致割合)がゼロになることを防ぐ
  return score/len(ja_texts) 

In [ ]:
ja_dev_texts = dataset_creator.Read(dev_ja_path, inp=True)
en_dev_texts = dataset_creator.Read(dev_en_path)
# t = beam_translate(ja_dev_texts[1])
score = Evaluate(ja_dev_texts, en_dev_texts)
print(f'BLUE_SCORE : {score}')

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/resource_loader.py:110: UserWarning: You are currently using TensorFlow 2.9.2 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.10.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops by setting environment variable `TF_ADDONS_PY_OPS=1` or using `tfa.options.disable_custom_kernel()` in your code. To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would need a TensorFlow version equal to or above 2.10.0 and strictly below 2.11.0.
 Note that nightly versions of TensorFlow, as well as non-pip TensorFlow li

BLUE_SCORE : 0.011338212573238514


In [ ]:
epoch = [1, 5, 10]
cotr = [0, 1.96, 3.4]
